In [1]:
%matplotlib inline

In [2]:
import numpy as np
import librosa, librosa.display
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as disp

In [3]:
import utils

In [4]:
good_tracks = utils.load('./good_tracks.csv')

/home/fx/598PS/fma/utils.py:56: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  'category', categories=SUBSETS, ordered=True)


In [5]:
good_tracks[('track', 'genre_top')].value_counts()

Rock                   14025
Experimental           10370
Electronic              9129
Hip-Hop                 3520
Folk                    2780
Instrumental            2051
Pop                     1994
International           1372
Classical               1220
Jazz                     568
Old-Time / Historic      547
Spoken                   413
Country                  194
Soul-RnB                 174
Blues                    110
Easy Listening            24
Name: (track, genre_top), dtype: int64

In [6]:
usable_genres = ['Rock', 'Electronic', 'Hip-Hop', 'Folk', 'Instrumental', 'Pop', 'Classical']
usable_tracks = good_tracks[good_tracks[('track', 'genre_top')]
            .isin(usable_genres)]

In [7]:
usable_tracks[('track', 'genre_top')].value_counts()

Rock            14025
Electronic       9129
Hip-Hop          3520
Folk             2780
Instrumental     2051
Pop              1994
Classical        1220
Name: (track, genre_top), dtype: int64

In [8]:
very_usable_tracks = usable_tracks[usable_tracks
                                   .apply(lambda s: len(s[('track', 'genres')]) == 1, axis=1, reduce=True)]

In [9]:
very_usable_tracks[('track', 'genre_top')].value_counts()

Rock            6653
Electronic      4092
Hip-Hop         3010
Folk            2113
Pop             1700
Instrumental    1317
Classical        969
Name: (track, genre_top), dtype: int64

In [10]:
very_usable_tracks[very_usable_tracks[('track', 'genre_top')] == 'Rock'].sample(n=900).index

Int64Index([146542, 139849,  19581,  19163,   3962,   1518,  63264,  62979,
            142294,   1079,
            ...
             58407, 151782,  57580, 105099,  84291,  75124,  14090, 136275,
             44287, 119254],
           dtype='int64', name='track_id', length=900)

In [11]:
genre_index = {}
for genre in usable_genres:
    genre_index[genre] = list(very_usable_tracks[very_usable_tracks[('track', 'genre_top')] == genre].sample(n=900).index)

In [12]:
for key in genre_index:
    print(key, len(genre_index[key]))

Classical 900
Hip-Hop 900
Pop 900
Rock 900
Electronic 900
Instrumental 900
Folk 900


In [ ]:
for genre in usable_genres:
    print(genre)
    for i in range(5):
        idx = genre_index[genre][i]
        f = utils.get_track_file(idx)
        print(f)
        disp.display(disp.Audio(f))
        y, fs = librosa.load(f)
        D = librosa.stft(y)
        librosa.display.specshow(librosa.amplitude_to_db(D, ref=np.max), y_axis='log', x_axis='time')
        plt.show()

In [13]:
train_size = 10
test_size = 2
train_y = []
test_y = []
for i in range(7):
    train_y.extend([i for j in range(train_size)])
    test_y.extend([i for k in range(test_size)])
train_y = np.array(train_y)
test_y = np.array(test_y)
print(test_y)

[0 0 1 1 2 2 3 3 4 4 5 5 6 6]


In [14]:
train_X = []
test_X = []
for genre in usable_genres:
    for i in range(train_size + test_size):
        idx = genre_index[genre][i]
        f = utils.get_track_file(idx)
        y = utils.load_music_fixed_length(f)
        print("data_{}".format(i))
        if i < train_size:
            train_X.append(y)
        else:
            test_X.append(y)
train_X = np.array(train_X)
test_X = np.array(test_X)
print(train_X.shape)

data_0
data_1
data_2
data_3
data_4
data_5
data_6
data_7
data_8
data_9
data_10
data_11
data_0
data_1
data_2
data_3
data_4
data_5
data_6
data_7
data_8
data_9
data_10
data_11
data_0
data_1
data_2
data_3
data_4
data_5
data_6
data_7
data_8
data_9
data_10
data_11
data_0
data_1
data_2
data_3
data_4
data_5
data_6
data_7
data_8
data_9
data_10
data_11
data_0
data_1
data_2
data_3
data_4
data_5
data_6
data_7
data_8
data_9
data_10
data_11
data_0
data_1
data_2
data_3
data_4
data_5
data_6
data_7
data_8
data_9
data_10
data_11
data_0
data_1
data_2
data_3
data_4
data_5
data_6
data_7
data_8
data_9
data_10
data_11
(70, 661500)


In [25]:
from librosa import feature
from histo_classifier import histo_classifier

# train_X = np.array([feature.mfcc(train_X[i]) for i in range(len(train_X))])
test_X = np.array([feature.mfcc(test_X[i]) for i in range(len(test_X))])
his_cla = histo_classifier()
his_cla.train(train_X, train_y, True)
preds = his_cla.predict(test_X, True)
print(preds)

TypeError: train() takes 3 positional arguments but 4 were given

In [20]:
preds = his_cla.predict(train_X)
print(preds)

[0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 4 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6]


In [21]:
print(np.sum(preds==train_y)/len(train_y))

0.971428571429
